# calculate rotation and transposition matrix

In [2]:
import os
data_folder = r'\\storm6-pc\Data\Pu\20211203-P_brain_CTP11-1000_CTP12DNA'
before_position_file = os.path.join(data_folder, 'positions_before.txt')
after_position_file = os.path.join(data_folder, 'positions_after.txt')


In [3]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [4]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [5]:
R, T

(array([[ 0.99999191, -0.00402355],
        [ 0.00402355,  0.99999191]]), array([-269.74914188,  904.55217609]))

# transpose 60x positions

In [6]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [7]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-4535.56999213 -2919.51504006]
 [-4536.41493735 -3129.51334021]
 [-4537.25988257 -3339.51164037]
 [-4538.10482779 -3549.50994052]
 [-4538.94977301 -3759.50824067]
 [-4539.79471822 -3969.50654083]
 [-4540.63966344 -4179.50484098]
 [-4541.48460866 -4389.50314114]
 [-4331.48630851 -4390.34808636]
 [-4330.64136329 -4180.3497862 ]
 [-4329.79641807 -3970.35148605]
 [-4328.95147285 -3760.35318589]
 [-4328.10652763 -3550.35488574]
 [-4327.26158241 -3340.35658558]
 [-4326.41663719 -3130.35828543]
 [-4325.57169197 -2920.35998528]
 [-4115.57339182 -2921.2049305 ]
 [-4116.41833704 -3131.20323065]
 [-4117.26328226 -3341.2015308 ]
 [-4118.10822748 -3551.19983096]
 [-4118.9531727  -3761.19813111]
 [-4119.79811792 -3971.19643127]
 [-4120.64306314 -4181.19473142]
 [-4121.48800836 -4391.19303157]
 [-3911.4897082  -4392.03797679]
 [-3910.64476298 -4182.03967664]
 [-3909.79981776 -3972.04137649]
 [-3908.95487254 -3762.04307633]
 [-3908.10992732 -3552.04477618]
 [-3907.2649821  -3342.04647602]
 [-3906.42

In [8]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\Data\Pu\20211203-P_brain_CTP11-1000_CTP12DNA\translated_positions_all.txt


# further adjust manually

In [19]:
new_centers = np.array([[-4535.57,-2919.52], [-4509.70,1858.76], [-632.27,-2623.83]])

adjust_new_centers = np.array([[-4516, -2910], [-4498.5, 1863.8], [-611.3, -2615.8]])

In [20]:
manual_shifts = adjust_new_centers - new_centers

In [21]:
manual_shift = np.round(np.mean(manual_shifts, axis=0), 2)

In [22]:
print(manual_shift)

[17.25  7.53]


In [23]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [24]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\Data\Pu\20211203-P_brain_CTP11-1000_CTP12DNA\adjusted_translated_positions_all.txt
